In [1]:
# load libraries
import json
import requests
from dotenv import dotenv_values
import sys
import os
import time
import re

# print cwd
os.getcwd()

'/Users/chasedawson/dev/uva_equity_center/climate_equity'

In [2]:
# send http request
def sendRequest(url, data, apiKey = None):
    json_data = json.dumps(data)
    
    if apiKey == None:
        response = requests.post(url, json_data)
    else:
        headers = {'X-Auth-Token': apiKey}
        response = requests.post(url, json_data, headers = headers)
        
        
    try:
        httpStatusCode = response.status_code
        
        if response == None:
            print("No output from service!")
            sys.exit()
            
        output = json.loads(response.text)
        if output['errorCode'] != None:
            print(output['errorCode'], "- ", output['errorMessage'])
            sys.exit()
            
        if httpStatusCode == 404:
            print("404 Not Found")
            sys.exit()
            
        elif httpStatusCode == 401:
            print("401 Unauthorized")
            sys.exit()
            
        elif httpStatusCode == 400:
            print("Error Code", httpStatusCode)
            sys.exit()
            
    except Exception as e:
        response.close()
        print(e)
        sys.exit()
    
    response.close()
    return output['data']
        

In [3]:
# logs-in using credentials and returns apiKey if successful
def login(username, password):
    serviceUrl = "https://m2m.cr.usgs.gov/api/api/json/stable/"

    # login information
    payload = {'username': username, 'password': password}

    # get apiKey 
    apiKey = sendRequest(serviceUrl + "login", payload)
    
    return apiKey

# call when finished working to ensure that no one else can use the API key
def logout(apiKey):
    serviceUrl = "https://m2m.cr.usgs.gov/api/api/json/stable/"
    endpoint = "logout"
    if sendRequest(serviceUrl + endpoint, None, apiKey) == None:
        print("Logged Out\n\n")
    else:
        print("Logout Failed\n\n")

In [4]:
# load username and password from .env
config = dotenv_values('.env')
apiKey = login(config["USGS_USERNAME"], config["USGS_PASSWORD"])

In [5]:
eastern_shore_bounds = {'upperRight': {'latitude': 38.08422, 'longitude': -75.07674},
                       'lowerLeft': {'latitude': 37.02165, 'longitude': -76.16713}}

cville_bounds = {'upperRight': {'latitude': 38.07493, 'longitude': -78.43942},
                'lowerLeft': {'latitude': 38.00855, 'longitude': -78.54259}}

In [6]:
# dataset-search
serviceUrl = "https://m2m.cr.usgs.gov/api/api/json/stable/"
endpoint = "dataset-search"

datasetName = "TIRS"
spatialFilter = {'filterType': 'mbr',
                'upperRight': cville_bounds['upperRight'],
                'lowerLeft': cville_bounds['lowerLeft']}
temporalFilter = {'start': '2020-06-21', 'end': '2020-09-21'}
payload = {'datasetName': datasetName,
          'spatialFilter': spatialFilter,
          'temporalFilter': temporalFilter}

datasets = sendRequest(serviceUrl + endpoint, payload, apiKey)

In [7]:
len(datasets)

4

In [8]:
for dataset in datasets:
    print(dataset['datasetAlias'])
    print(dataset['abstractText'])
    print('\n')

landsat_8_c1
The USGS Earth Resources Observation and Science (EROS) Center archive holds data collected by the Landsat suite of satellites, beginning with Landsat 1 in 1972.  Landsat 8, launched February 11, 2013, is the latest Landsat sensor.


lsr_landsat_8_c1
The USGS Earth Resources Observation and Science (EROS) Center archive holds data collected by the Landsat suite of satellites, beginning with Landsat 1 in 1972.  Landsat 8, launched February 11, 2013, is the latest Landsat sensor.


landsat_ot_c2_l1
The USGS Earth Resources Observation and Science (EROS) Center archive holds data collected by the Landsat suite of satellites, beginning with Landsat 1 in 1972.  Landsat 8, launched February 11, 2013, is the latest Landsat sensor.


landsat_ot_c2_l2
The USGS Earth Resources Observation and Science (EROS) Center archive holds data collected by the Landsat suite of satellites, beginning with Landsat 1 in 1972.  Landsat 8, launched February 11, 2013, is the latest Landsat sensor.




In [9]:
datasetName = "landsat_ot_c2_l2"
payload = {'datasetName': datasetName}
dataset = sendRequest(serviceUrl + "dataset", payload, apiKey)
dataset

{'abstractText': 'The USGS Earth Resources Observation and Science (EROS) Center archive holds data collected by the Landsat suite of satellites, beginning with Landsat 1 in 1972.  Landsat 8, launched February 11, 2013, is the latest Landsat sensor.',
 'acquisitionStart': '2013-04-11',
 'acquisitionEnd': None,
 'catalogs': ['EE'],
 'collectionName': 'Landsat 8 OLI/TIRS C2 L2',
 'collectionLongName': 'Landsat 8 Operational Land Imager and Thermal Infrared Sensor Collection 2 Level-2',
 'datasetId': '5e83d14f2fc39685',
 'datasetAlias': 'landsat_ot_c2_l2',
 'datasetCategoryName': 'Landsat Collection 2 Level-2',
 'dataOwner': 'LSAA',
 'dateUpdated': '2020-12-02 16:29:18.381513-06',
 'doiNumber': 'https://doi.org/10.5066/P9OGBGM6 ',
 'ingestFrequency': 'P1D',
 'keywords': 'None,Visible Wavelengths,Earth Science,Infrared Wavelengths,Infrared Imagery,Imagery ,Earth Resources Observation and Science (EROS) Center,Coastal Aerosol ,Visible Imagery,U.S. Geological Survey (USGS),Thermal Infrared S

In [10]:
# search and find scenes in landsat_ot_c2_l2 that I can download
payload = {
    'datasetName': datasetName,
    'maxResults': 4,
    'startingNumber': 1,
    'sceneFilter': {
        'spatialFilter': {
            'filterType': 'mbr',
            'lowerLeft': cville_bounds['lowerLeft'],
            'upperRight': cville_bounds['upperRight']
        },
        'acquisitionFilter': {
            'start': '2020-06-21',
            'end': '2020-09-21'
        },
        'cloudCoverFilter': {
            'max': 10,
            'min': 0,
            'includeUnknown': False
        }
    }
}

print("Searching Scenes...")
scenes = sendRequest(serviceUrl + "scene-search", payload, apiKey)
print("Found {num_scenes} Scene(s)".format(num_scenes = scenes['recordsReturned']))

Searching Scenes...
Found 4 Scene(s)


In [11]:
sceneIds = []
for result in scenes['results']:
    sceneIds.append(result['entityId'])
    
payload = {'datasetName': datasetName, 'entityIds': sceneIds}
downloadOptions = sendRequest(serviceUrl + "download-options", payload, apiKey)

# Aggregate a list of available products
downloads = []
for product in downloadOptions:
    # make sure the product is available for this scene
    if product['available'] == True:
        downloads.append({'entityId': product['entityId'],
                         'productId': product['id']})

In [12]:
if downloads: 
    requestedDownloadsCount = len(downloads)
    # set a label for the download request
    label = 'cville-summer-2020-download'
    payload = {'downloads': downloads, 
              'label': label}
    requestResults = sendRequest(serviceUrl + "download-request", payload, apiKey)

In [13]:
# use content-disposition to get filename
def getFilename_fromCd(cd):
    if not cd:
        return None
    fname = re.findall('filename=(.+)', cd)
    if len(fname) == 0:
        return None
    
    return re.sub('\"', '', fname[0]) # remove extra quotes

def download_file(link):
    res = requests.get(link)
    filename = getFilename_fromCd(res.headers.get('content-disposition'))
    open(filename, 'wb').write(res.content)
    

In [15]:
# after submitting, preparingDownloads has a valid link that can be used but data may 
# not be immediately available
if requestResults['preparingDownloads'] != None and len(requestResults['preparingDownloads']) > 0:
    payload = {'label': label}  
    moreDownloadUrls = sendRequest(serviceUrl + "download-retrieve", payload, apiKey)
    downloadIds = []
    for download in moreDownloadUrls['available']:
        downloadIds.append(download['downloadId'])
        print("DOWNLOAD: " + download['url'])
    
    for download in moreDownloadUrls['requested']:
        downloadIds.append(download['downloadId'])
        print("DOWNLOAD: " + download['url'])
        
    # didn't get all requested downloads, call the download-retrieve endpoint again
    while len(downloadIds) < requestedDownloadsCount:
        preparingDownloads = requestedDownloadsCount - len(downloadIds)
        print('\n', preparingDownloads, "download(s) are not available. Waiting for 30 seconds.\n")
        time.sleep(30)
        print("Trying to retrieve data\n")
        moreDownloadUrls = sendRequest(serviceUrl + "download-retrieve", payload, apiKey)
        for download in moreDownloadUrls['available']:
            if download['downloadId'] not in downloadIds:
                downloadIds.append(download['downloadId'])
                print("DOWNLOAD: " + download['url'])
else:
    # get all available downloads
    for download in requestResults['availableDownloads']:
        link = download['url']
        print("DOWNLOADING: " + link)
        download_file(link)

        # download_file(download['url'])
    print("\nAll files have been downloaded.\n")

DOWNLOADING: https://dds.cr.usgs.gov/download/eyJpZCI6NDgzNTE5NDgsImNvbnRhY3RJZCI6MjYwOTAwNjN9
DOWNLOADING: https://dds.cr.usgs.gov/download/eyJpZCI6NDgzNTE5NDksImNvbnRhY3RJZCI6MjYwOTAwNjN9
DOWNLOADING: https://dds.cr.usgs.gov/download/eyJpZCI6NDgzNTAxODEsImNvbnRhY3RJZCI6MjYwOTAwNjN9
DOWNLOADING: https://dds.cr.usgs.gov/download/eyJpZCI6NDgzNTAxODIsImNvbnRhY3RJZCI6MjYwOTAwNjN9

All files have been downloaded.



In [16]:
# References
# [Downloading Files From Web Using Python](https://www.tutorialspoint.com/downloading-files-from-web-using-python)
# [Jupyter Tips and Tricks](https://chrieke.medium.com/jupyter-tips-and-tricks-994fdddb2057)
# [USGS/EROS Inventory Service Documentation (Machine-to-Machine) API](https://m2m.cr.usgs.gov/api/docs/json/#section-overview)
